In [2]:
library(tidyverse)
library(repr)
library(rvest)
library(stringr)
library(dbplyr)
library(DBI)
library(RPostgres)
library(lubridate)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘rvest’ was built under R version 4.0.2”
Loading required package: xml2


Attaching package: ‘rvest’


The following object is masked from ‘package:purrr’:

    pluck


The following object is masked from ‘package:readr’:

    guess_encoding



Attaching package: ‘dbplyr’


The following objects are masked from ‘pac

In [3]:
# Title: Factors Contributing to Algerian Forest Fires
# Introduction:
forest_fires_untidy <- read_csv(file = "https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv", 
                                skip =1, col_names = TRUE, col_types = cols_only(
                                      day = col_integer(),
                                      month = col_integer(),
                                      year = col_integer(),
                                      Temperature = col_double(),
                                      RH = col_double(),
                                      Ws = col_double(),
                                      Rain = col_double(),
                                      FFMC = col_double(),
                                      DMC = col_double(),
                                      DC = col_double(),
                                      ISI = col_double(),
                                      BUI = col_double(),
                                      FWI = col_double(),
                                      Classes = col_character()))  

bejaia_region <- forest_fires_untidy %>%
    slice(1:122) %>%
    mutate(region = "bejaia")

sidi_bel_region <- forest_fires_untidy %>%
    slice(125:246) %>%
    mutate(region = "sidi bel abbes")

forest_fire_data <- rbind(bejaia_region, sidi_bel_region) 

fire_data <- forest_fire_data %>%
     select(day, month, Temperature, FWI, Classes)

fire_data

set.seed(1)
fire_split <- initial_split(fire_data, prop = 0.75, strata = Classes)
fire_train <- training(fire_split)
fire_test <- testing(fire_split)

fire_recipe <- recipe(Classes ~ month + Temperature + FWI, data = fire_train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())

set.seed(1)
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) %>%
    set_engine("kknn") %>%
    set_mode("classification")

knn_fit <- workflow() %>%
    add_recipe(fire_recipe) %>%
    add_model(knn_spec) %>%
    fit(data = fire_train)

fire_test_predictions <- predict(knn_fit, fire_test) %>%
    bind_cols(fire_test)

Warning message:
“18 parsing failures.
row   col   expected                        actual                                                                                                       file
123 day   an integer Sidi-Bel Abbes Region Dataset 'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
123 NA    14 columns 1 columns                     'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 day   an integer day                           'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 month an integer month                         'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
124 year  an integer year                          'https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv'
... ..... ...

day,month,Temperature,FWI,Classes
<int>,<int>,<dbl>,<dbl>,<chr>
1,6,29,0.5,not fire
2,6,29,0.4,not fire
3,6,26,0.1,not fire
⋮,⋮,⋮,⋮,⋮
28,9,27,0.2,not fire
29,9,24,0.7,not fire
30,9,24,0.5,not fire


ERROR: Error in initial_split(fire_data, prop = 0.75, strata = Classes): could not find function "initial_split"
